<a href="https://colab.research.google.com/github/JMQuehl/stable-diffusion/blob/colab/colab_notebooks/Stable_Diffusion_All_in_One.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Stable Diffusion** 
In order to use this colab, first get an access token for the stable diffusion weights hosted on huggingface (https://huggingface.co/CompVis/stable-diffusion). If you accepted the license for this model and generated a key, put that key into the access_token field below.

If you get cuda_out_of_memory or similar errors, try using the lower precision model. There is an (optional) SFW filter, which you can enable below in order to ensure that no offensive content might be created.

In [ ]:
access_token = "put_your_token_here" #@param{type: 'string'}
if not access_token.startswith("hf_"):
  from huggingface_hub import notebook_login
  notebook_login()
  access_token = True
use_lower_precision = False #@param{type: 'boolean'}
use_pipeline_with_safety_filter = False #@param{type: 'boolean'}
#@markdown If you connect your Google Drive, you can save the final image of each run on your drive.
use_google_drive = True #@param{type: 'boolean'}

# 1. Setup

First, please make sure you are using a GPU runtime to run this notebook, so inference is much faster. If the following command fails, use the `Runtime` menu above and select `Change runtime type`.

In [ ]:
!nvidia-smi

In [ ]:
import os
def createPath(filepath):
    os.makedirs(filepath, exist_ok=True)

if use_google_drive: 
  try:
      from google.colab import drive
      print("Google Colab detected. Using Google Drive.")
      is_colab = True
      #@markdown Click here if you'd like to save the diffusion model checkpoint file to (and/or load from) your Google Drive:
      save_models_to_google_drive = True #@param {type:"boolean"}
  except:
      is_colab = False
      use_google_drive = False
      save_models_to_google_drive = False
      print("Google Colab not detected.")

if is_colab:
    if use_google_drive :
        drive.mount('/content/drive')
        root_path = '/content/drive/MyDrive/AI/Stable_Diffusion'
    else:
        root_path = '/content'
else:
    root_path = os.getcwd()


outDirPath = f'{root_path}/images_out'
createPath(outDirPath)

if is_colab:
    if (use_google_drive and not save_models_to_google_drive) or not use_google_drive:
        model_path = '/content/models'
        createPath(model_path)
    if use_google_drive and save_models_to_google_drive:
        model_path = f'{root_path}/models'
        createPath(model_path)
else:
    model_path = f'{root_path}/models'
    createPath(model_path)

Prepare and set up libraries for Stable Diffusion

In [ ]:
!pip install diffusers==0.2.4
!pip install transformers scipy ftfy gradio datasets tqdm
!pip install "ipywidgets>=7,<8"
!wget https://raw.githubusercontent.com/huggingface/diffusers/4674fdf807cdefd4db1758067c0207872d805f8c/examples/inference/image_to_image.py

from google.colab import output
output.enable_custom_widget_manager()

Prepare Stable Diffusion Pipelines for Text2Image and Image2Image

In [ ]:
import torch
import gc
from diffusers import StableDiffusionPipeline
from image_to_image import StableDiffusionImg2ImgPipeline, preprocess
from huggingface_hub import snapshot_download

stable_diff_model_folder = f'{model_path}/stable-diffusion-v1-4'

if use_lower_precision:
  hf_revision = "fp16"
  torch_type = torch.float16
else:
  hf_revision = None
  torch_type = None

snapshot_download(repo_id="CompVis/stable-diffusion-v1-4", cache_dir=stable_diff_model_folder, use_auth_token=access_token)
print ("Download completed.")

pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", cache_dir=stable_diff_model_folder, revision=hf_revision, torch_dtype=torch_type, use_auth_token=access_token) 
im2im_pipe = StableDiffusionImg2ImgPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", cache_dir=stable_diff_model_folder, revision=hf_revision, torch_dtype=torch_type, use_auth_token=access_token)

if not use_pipeline_with_safety_filter:
  def dummy_checker(images, **kwargs): return images, False
  pipe.safety_checker = dummy_checker
  im2im_pipe.safety_checker = dummy_checker

pipe = pipe.to("cuda")
im2im_pipe = im2im_pipe.to("cuda")

Define Helper Function(s)

In [ ]:
from PIL import Image
def image_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size
    
    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid

def get_free_filename(folder, base_file_name, extension="png"):
  i = 0
  i_string = str(i).rjust(4, "0")
  while os.path.exists(os.path.join(folder, f"{base_file_name}_{i_string}.{extension}")):
    i += 1
    i_string = str(i).rjust(4, "0")
  return f"{base_file_name}_{i_string}", os.path.join(folder, f"{base_file_name}_{i_string}.{extension}")






# 2. Settings

In [ ]:
import os
from google.colab import files
import shutil
#@markdown #####**General-Settings:**
batch_name ="Portals" #@param{type: 'string'}
prompt = "A portal to the depths of hell, octane render, 4k" #@param{type: 'string'}
width = 512 #@param{type: 'number'}
height = 512 #@param{type: 'number'}
steps = 50 #@param{type: 'number'}
use_seed = False #@param{type: 'boolean'}
seed = 0 #@param{type: 'number'}
guidance_scale = 7.5 #@param{type: 'number'}
num_cols = 2 #@param{type: 'number'}
num_rows = 2 #@param{type: 'number'}
batch_size = num_rows * num_cols

#@markdown #####**Init_Image-Settings:**
use_init_image = False #@param{type: 'boolean'}
init_image_strength = 0.9 #@param{type: 'number'}
upload_new_image = False #@param{type: 'boolean'}
#@markdown if a new image is uploaded that takes precedence over the batch image.
use_image_from_previous_batch_as_init = True #@param{type: 'boolean'}
batch_image_number = 0 #@param{type: 'number'}
resize = False #@param{type: 'boolean'}

file_extension = "png" #@param{type: 'string'}

if height > 512 or width > 512:
  print("Warning: If height or width are larger than 512 coherence might be lost!")

generator = torch.Generator("cuda")

if use_init_image :
  if use_image_from_previous_batch_as_init and "all_images" in globals():
    init_image = all_images[batch_image_number]

  if not "init_image" in globals() or upload_new_image: 
    upload_folder = 'upload'
    if os.path.isdir(upload_folder):
        shutil.rmtree(upload_folder)
    os.mkdir(upload_folder)
    # upload images
    uploaded = files.upload()

    for filename in uploaded.keys() :
      dst_path = os.path.join(upload_folder, filename)
      print(f'move {filename} to {dst_path}')
      shutil.move(filename, dst_path)
    filename = os.path.join(upload_folder, list(uploaded.keys())[0])
    init_image = Image.open(filename).convert("RGB")


  if (resize):
    init_image = init_image.resize((width, height))
  init_image_cuda = preprocess(init_image)
  print ("Init image will be: ")
  display(init_image)

# 3. Run Standard Pipeline

If using a seed this image will always be the same for the same *prompt*



In [ ]:
gc.collect()
torch.cuda.empty_cache()
from torch import autocast

if not use_seed :
  import random
  random.seed()
  seed = random.randint(0, 2**32)

generator = torch.Generator("cuda").manual_seed(seed)
print("using seed: {seed}".format(seed = seed))

prompts = [prompt] * num_cols

all_images = []
for i in range(num_rows):
  with autocast("cuda"):
    if not use_init_image or "init_image_cuda" not in globals():
      images = pipe(prompts, num_inference_steps=steps, height=height, width=width, generator=generator, guidance_scale=guidance_scale)["sample"]  # image here is in [PIL format](https://pillow.readthedocs.io/en/stable/)
    else:
      images = im2im_pipe(prompts, num_inference_steps=steps, generator=generator, guidance_scale=guidance_scale, init_image=init_image_cuda, strength=init_image_strength)["sample"]
  all_images.extend(images)

grid = image_grid(all_images, rows=num_rows, cols=num_cols)
grid

If you want to save any images, run the following cell

In [ ]:
only_save_image_with_number = 3 #@param{type: 'number'}
save_all_images = False #@param{type: 'boolean'}

out_batch_path = os.path.join(outDirPath, batch_name)
createPath(out_batch_path)

if not save_all_images and 0 <= only_save_image_with_number < len(all_images):
  out_file_name, full_out_file_path = get_free_filename(out_batch_path, hash(prompt), file_extension)
elif save_all_images:
  for image in all_images:
    out_file_name, full_out_file_path = get_free_filename(out_batch_path, hash(prompt), file_extension)
    prompt_file = open(os.path.join(out_batch_path, str(hash(prompt)) + "_prompt.txt"), 'w')
    prompt_file.write(prompt)
    prompt_file.close()

    print (f"saving picture to : {full_out_file_path}")
    image.save(full_out_file_path)
else:
  print("saving no picture...")

# 4. Prepare ESRGAN for upscaling an image
Set Upscaler Settings

In [ ]:
image_to_enhance = 3 #@param{type: 'number'}
pil_img = all_images[image_to_enhance]

esrgan_model_name = "RealESRGAN_x4plus" #@param ["RealESRGAN_x4plus", "RealESRNet_x4plus", "RealESRGAN_x4plus_anime_6B", "RealESRGAN_x2plus", "realesr-animevideov3"]
outscale = 2 #@param{type: 'number'}
face_enhance = True #@param{type: 'boolean'}

esrgan_out_batch_path = os.path.join(outDirPath, f"{batch_name}_ESRGAN")
createPath(esrgan_out_batch_path)

out_file_name, full_out_file_path = get_free_filename(esrgan_out_batch_path, hash(prompt), file_extension)
prompt_file = open(os.path.join(esrgan_out_batch_path, str(hash(prompt)) + "_prompt.txt"), 'w')
prompt_file.write(prompt)
prompt_file.close()

print (f"saving picture to : {full_out_file_path}")

all_images[image_to_enhance].save(full_out_file_path)
all_images[image_to_enhance]

In [ ]:
if not "esrgan_initialized" in globals():
  # Clone Real-ESRGAN and enter the Real-ESRGAN
  !git clone https://github.com/xinntao/Real-ESRGAN.git
  %cd Real-ESRGAN
  # Set up the environment
  !pip install basicsr
  !pip install facexlib
  !pip install gfpgan
  !pip install -r Real-ESRGAN/requirements.txt
  !python setup.py develop
  # Download the pre-trained model  
  !wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -P '{model_path}/ESRGAN_models' -nc
  !wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.1/RealESRNet_x4plus.pth -P '{model_path}/ESRGAN_models' -nc
  !wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.2.4/RealESRGAN_x4plus_anime_6B.pth -P '{model_path}/ESRGAN_models' -nc
  !wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.1/RealESRGAN_x2plus.pth -P '{model_path}/ESRGAN_models' -nc
  !wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.5.0/realesr-animevideov3.pth -P '{model_path}/ESRGAN_models' -nc
  !wget https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth -P '{model_path}/ESRGAN_models' -nc
  !cp {model_path}/ESRGAN_models/* 'experiments/pretrained_models'

  esrgan_initialized = True
  %cd ..


In [ ]:
gc.collect()
torch.cuda.empty_cache()
%cd Real-ESRGAN
if face_enhance:
  !python inference_realesrgan.py -n RealESRGAN_x4plus -i {full_out_file_path} --outscale {outscale} -o {esrgan_out_batch_path} --face_enhance
else:
  !python inference_realesrgan.py -n RealESRGAN_x4plus -i {full_out_file_path} --outscale {outscale} -o {esrgan_out_batch_path}
%cd ..

shutil.move(os.path.join(esrgan_out_batch_path, f"{out_file_name}_out.{file_extension}"), os.path.join(esrgan_out_batch_path, f"{out_file_name}_upscaled.{file_extension}"))
upscaled_image = Image.open(os.path.join(esrgan_out_batch_path, f"{out_file_name}_upscaled.{file_extension}"))
upscaled_image